In [ ]:
from kafka import KafkaProducer
import yfinance as yf
import json
import time
from datetime import datetime, timedelta

# Inisialisasi Kafka Producer
producer = KafkaProducer(bootstrap_servers='localhost:9092')

# Fungsi untuk mengirim data saham ke Kafka
def send_stock_data(symbol, data):
    producer.send('stock-data', json.dumps(data).encode('utf-8'))

# Mendapatkan data saham BCA dari Yahoo Finance
def get_stock_data():
    symbol = 'BBCA.JK'  # Simbol saham BCA di bursa Indonesia
    end_date = datetime.now().strftime('%Y-%m-%d')  # Tanggal hari ini
    start_date = (datetime.now() - timedelta(days=5*365)).strftime('%Y-%m-%d')  # Tanggal 5 tahun terakhir
    stock = yf.Ticker(symbol)
    data = stock.history(start=start_date, end=end_date)  # Mendapatkan data tiap bulan hingga saat ini
    for _, row in data.iterrows():
        stock_data = {
            'date': row.name.strftime('%Y-%m-%d'),  # Format tanggal menjadi 'YYYY-MM-DD'
            'open': row['Open'],
            'high': row['High'],
            'low': row['Low'],
            'close': row['Close'],
            'volume': row['Volume']
        }
        send_stock_data(symbol, stock_data)
        time.sleep(1)  # Delay 1 detik antara pengiriman data

# Menjalankan pengambilan data saham sekali
get_stock_data()

# Tutup koneksi Kafka setelah selesai
producer.close()